In [2]:
!pip install Mastodon.py confluent-kafka beautifulsoup4 lxml requests


In [6]:
import json
from mastodon import Mastodon, StreamListener
from confluent_kafka import Producer, KafkaException
from confluent_kafka.admin import AdminClient, NewTopic
from datetime import datetime
from bs4 import BeautifulSoup
import logging

# Configurer le journal pour capturer les erreurs et les informations
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Configurer Mastodon avec les identifiants de l'application
mastodon = Mastodon(
    client_id='izb9ti8Xlag6iauwHUAVORXSjL-l554tsFaZvU4RDII',
    client_secret='kXrzsq8fXTZEyIFbu1mcRCfDWY-8_ZJcLmaWJQRSGLg',
    access_token='wbhGh7WATwQJ2uHuWnBZ5rQ9SD-idQdk2O93_68Gvis',
    api_base_url='https://mastodon.social'
)

# Configurer Kafka AdminClient pour créer le sujet Kafka
admin_client_conf = {'bootstrap.servers': 'kafka:9092'}
admin_client = AdminClient(admin_client_conf)

topic_name = 'mastodon_stream'

# Créer un nouveau sujet Kafka si nécessaire
new_topic = NewTopic(topic_name, num_partitions=1, replication_factor=1)

try:
    fs = admin_client.create_topics([new_topic])
    for topic, f in fs.items():
        try:
            f.result()  # Attendre la création du sujet
            logger.info(f"Sujet '{topic}' créé avec succès.")
        except KafkaException as e:
            if e.args[0].code() == 36:  # Vérifier si le code d'erreur est TOPIC_ALREADY_EXISTS (val=36)
                logger.info(f"Sujet '{topic}' existe déjà.")
            else:
                logger.error(f"Erreur lors de la création du sujet '{topic}': {e}")
except Exception as e:
    logger.error(f"Erreur inconnue lors de la création du sujet: {e}")

# Configurer Kafka Producer avec confluent-kafka
producer_conf = {'bootstrap.servers': 'kafka:9092'}
producer = Producer(producer_conf)


# Fonction personnalisée pour sérialiser les objets datetime
def datetime_converter(obj):
    if isinstance(obj, datetime):
        return obj.isoformat()
    raise TypeError("Type not serializable")


# Fonction de rapport de livraison
def delivery_report(err, msg):
    if err is not None:
        logger.error(f"Erreur lors de la livraison du message: {err}")
    else:
        logger.info(f"Message envoyé: {msg.topic()} [{msg.partition()}] @ offset {msg.offset()}")


# Créer une classe StreamListener pour écouter les toots en temps réel
class TootStreamListener(StreamListener):
    def on_update(self, toot):
        try:
            # Extraire le texte du toot en supprimant les balises HTML
            toot_content = BeautifulSoup(toot['content'], 'html.parser').get_text(strip=True)

            # Filtrer les toots par mots-clés spécifiques
            keywords = [
                "#AI", "#ArtificialIntelligence", "#MachineLearning", "#DeepLearning",
                "#NLP", "#ReinforcementLearning", "#ComputerVision", "#NeuralNetworks",
                "#GenerativeAI", "#SupervisedLearning", "#UnsupervisedLearning",
                "#TensorFlow", "#PyTorch", "#OpenAI", "#HuggingFace", "#ChatGPT",
                "#GPT4", "#MLOps", "#DataEngineering", "#Robotics", "#SelfDrivingCars",
                "#AIArt", "#AIinHealthcare", "#AIAssistant", "#Automation",
                "#RecommendationSystems", "#SpeechRecognition", "Sam Altman",
                "Elon Musk", "Andrew Ng", "#AIEthics", "#AIConference", "#AIResearch", "Chat gpt", "chatgpt",
                "Sam altman"
            ]

            if any(keyword in toot_content.lower() for keyword in keywords):
                # Structurer les données
                toot_structured = {
                    'id': toot.get('id'),
                    'created_at': toot.get('created_at'),
                    'content': toot_content,
                    'user_id': toot.get('account', {}).get('id'),
                    'username': toot.get('account', {}).get('username'),
                    'acct': toot.get('account', {}).get('acct'),
                    'display_name': toot.get('account', {}).get('display_name'),
                    'visibility': toot.get('visibility'),
                    'reblogs_count': toot.get('reblogs_count'),
                    'favourites_count': toot.get('favourites_count'),
                    'tags': [tag.get('name') for tag in toot.get('tags', [])],
                    'language': toot.get('language'),
                    'replies_count': toot.get('replies_count'),
                    'followers_count': toot.get('account', {}).get('followers_count'),
                    'following_count': toot.get('account', {}).get('following_count'),
                    'statuses_count': toot.get('account', {}).get('statuses_count')
                }

                # Sérialiser le toot structuré en JSON
                toot_value = json.dumps(toot_structured, default=datetime_converter, ensure_ascii=False)

                # Envoyer le toot vers Kafka
                producer.produce(topic_name, value=toot_value, callback=delivery_report)
                producer.flush()
                logger.info("Toot structuré envoyé à Kafka: %s", toot_structured)
        except Exception as e:
            logger.error(f"Erreur lors de l'envoi du toot à Kafka: {e}")


# Démarrer le streaming public pour écouter les toots en temps réel
listener = TootStreamListener()
mastodon.stream_public(listener)


INFO:__main__:Sujet 'mastodon_stream' existe déjà.


SSLError: HTTPSConnectionPool(host='streaming.mastodon.social', port=443): Max retries exceeded with url: /api/v1/streaming/public (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1006)')))